In [7]:
import yfinance as yf
import pandas as pd

dataF = yf.download("USDCAD=X", start="2024-11-15", end="2024-12-29", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,USDCAD=X,USDCAD=X,USDCAD=X,USDCAD=X,USDCAD=X,USDCAD=X
Datetime,,,,,,
2024-11-15 00:00:00,1.40630,1.40630,1.40650,1.4059,1.40612,0
2024-11-15 00:15:00,1.40620,1.40620,1.40661,1.4059,1.40630,0
2024-11-15 00:30:00,1.40688,1.40688,1.40698,1.4062,1.40645,0
2024-11-15 00:45:00,1.40642,1.40642,1.40694,1.4061,1.40686,0
2024-11-15 01:00:00,1.40594,1.40594,1.40651,1.4056,1.40637,0
...,...,...,...,...,...,...
2024-12-27 21:00:00,1.44151,1.44151,1.44182,1.4410,1.44166,0
2024-12-27 21:15:00,1.44150,1.44150,1.44190,1.4410,1.44153,0


In [8]:
def signal_generator(df):
    open = df.Open.iloc[-1].item()
    close = df.Close.iloc[-1].item()
    previous_open = df.Open.iloc[-2].item()
    previous_close = df.Close.iloc[-2].item()
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [9]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    2584
1     170
2     161
Name: count, dtype: int64

In [11]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient   
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [12]:
access_token = "1248d6c07c72e767c5bd9aead28278d6-30afd9779ea269c077b9759ffd5945d9" # from Oanda Account
accountID = "101-001-30615097-001"

# API Client
client = API(access_token)

In [13]:
def get_candles(n):
# Using Yahoo Finance to fetch Apple stock data
    #import yfinance as yf
    #dataF = yf.download("AAPL", interval="15m", period="1d")
    #return dataF.tail(n)

    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.USD_CAD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
   print(float(str(candle.bid.o))>1)


True
True
True


In [14]:
def trading_job():
    candles = get_candles(3)
    #dfstream = candles.copy()
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
         
    SLTPRatio = 2.0
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    #SLBuy = dfstream['Close'].iloc[-1] - previous_candleR
    #SLSell = dfstream['Close'].iloc[-1] + previous_candleR

    #TPBuy = dfstream['Close'].iloc[-1] + previous_candleR * SLTPRatio
    #TPSell = dfstream['Close'].iloc[-1] - previous_candleR * SLTPRatio

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)

    #print(dfstream.iloc[:-1, :])
    #print(f"TPBuy: {TPBuy}, SLBuy: {SLBuy}, TPSell: {TPSell}, SLSell: {SLSell}")

    # signal = 2

    #if signal == 1:  # Sell
    #    mo = MarketOrderRequest(
    #        instrument="AAPL.US",  # Replace with Apple's instrument on OANDA
    #        units=-10,  # Adjust the units accordingly
    #        takeProfitOnFill=TakeProfitDetails(price=TPSell).data,
    #        stopLossOnFill=StopLossDetails(price=SLSell).data,
    #    )
    #    r = orders.OrderCreate(accountID, data=mo.data)
    #    rv = client.request(r)
    #    print(rv)
    #elif signal == 2:  # Buy
    #    mo = MarketOrderRequest(
    #        instrument="AAPL.US",  # Replace with Apple's instrument on OANDA
    #        units=10,  # Adjust the units accordingly
    #        takeProfitOnFill=TakeProfitDetails(price=TPBuy).data,
    #        stopLossOnFill=StopLossDetails(price=SLBuy).data,
    #    )
    #    r = orders.OrderCreate(accountID, data=mo.data)
    #    rv = client.request(r)
    #    print(rv)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="USD_CAD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="USD_CAD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [15]:
trading_job()

#scheduler = BlockingScheduler(timezone="America/Los_Angeles")
#scheduler.add_job(trading_job, 'date', run_date='2024-12-19 01:39:00', timezone="America/Los_Angeles")
# scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='01-15', minute='1,16,31,46')
#scheduler.start()

      Open    Close    High      Low
0  1.43465  1.43487  1.4349  1.43462
1  1.43487  1.43486  1.4351  1.43484
1.4353900000000004    1.43461    1.4343499999999998    1.4351300000000002
{'orderCreateTransaction': {'id': '64', 'accountID': '101-001-30615097-001', 'userID': 30615097, 'batchID': '64', 'requestID': '61322905040365808', 'time': '2024-12-31T04:06:38.685421369Z', 'type': 'MARKET_ORDER', 'instrument': 'USD_CAD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.43539', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.43461', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '65', 'accountID': '101-001-30615097-001', 'userID': 30615097, 'batchID': '64', 'requestID': '61322905040365808', 'time': '2024-12-31T04:06:38.685421369Z', 'type': 'ORDER_FILL', 'orderID': '64', 'instrument': 'USD_CAD', 'units': '1000', 'requestedUnits': '1000', 'price': '1.43503', 'pl': '0.0000